In [1]:
import numpy as np
import cv2
%matplotlib inline
import matplotlib.pyplot as plt

In [71]:
tot_video_cnt = 0
width = 400
height = 240

ds_x = []
ds_y = [] 

ts_x = []
ts_y = [] 

class wildfire:

    #num_pt : 1 => fire, 0 => no_fire
    #cnt : frame cut count
    #FileName : Video file name 
    def generate_dataset(main,frCnt,num_pt,capture_cnt,FileName):
        
        cap_cnt = 0
        frame_cnt = 0 #프레임갯수
        cap = cv2.VideoCapture(FileName)
        mog = cv2.createBackgroundSubtractorMOG2()
        
        while True:
        
            if cap_cnt == capture_cnt:
                break
            print("filename:{}, cnt:{}".format(FileName,cap_cnt+1)) 
            
            frame_cnt += 1 # 프레임 갯수
            
            if (frame_cnt%frCnt) == 0:             
            
                ret, frame = cap.read()
                if ret == False:
                    break

                dst = cv2.fastNlMeansDenoisingColored(frame,None,25,25,7,21)    
                fgmask = mog.apply(dst)
                ds_x.append(fgmask)
                ds_y.append(num_pt)

                cap_cnt += 1

        cap.release()      

    #num_pt : 1 => fire, 0 => no_fire
    #cnt : frame cut count
    #FileName : Video file name 
    def test_dataset(main,frCnt,num_pt,capture_cnt,FileName):
        
        cap_cnt = 0
        frame_cnt = 0 #프레임갯수
        cap = cv2.VideoCapture(FileName)
        mog = cv2.createBackgroundSubtractorMOG2()
        
        while True:
        
            if cap_cnt == capture_cnt:
                break
            print("filename:{}, cnt:{}".format(FileName,cap_cnt+1)) 
            
            frame_cnt += 1 # 프레임 갯수
            
            if (frame_cnt%frCnt) == 0:             
            
                ret, frame = cap.read()
                if ret == False:
                    break

                dst = cv2.fastNlMeansDenoisingColored(frame,None,25,25,7,21)    
                fgmask = mog.apply(dst)
                ts_x.append(fgmask)
                ts_y.append(num_pt)

                cap_cnt += 1

        cap.release()   

In [72]:
wf = wildfire()
video_cnt = 10
wf.generate_dataset(1,20,video_cnt,'dalma_400240.mp4') # param : 메인변수, 건너뛸 프레임갯수, 캡처화면수, video명
tot_video_cnt += video_cnt

filename:dalma_400240.mp4, cnt:1
filename:dalma_400240.mp4, cnt:2
filename:dalma_400240.mp4, cnt:3
filename:dalma_400240.mp4, cnt:4
filename:dalma_400240.mp4, cnt:5
filename:dalma_400240.mp4, cnt:6
filename:dalma_400240.mp4, cnt:7
filename:dalma_400240.mp4, cnt:8
filename:dalma_400240.mp4, cnt:9
filename:dalma_400240.mp4, cnt:10


In [76]:
video_cnt = 30
wf.generate_dataset(1,80,video_cnt,'gwanak_400240.mp4') # param : 메인변수, 건너뛸 프레임갯수, 캡처화면수, video명
tot_video_cnt += video_cnt

filename:gwanak_400240.mp4, cnt:1
filename:gwanak_400240.mp4, cnt:2
filename:gwanak_400240.mp4, cnt:3
filename:gwanak_400240.mp4, cnt:4
filename:gwanak_400240.mp4, cnt:5
filename:gwanak_400240.mp4, cnt:6
filename:gwanak_400240.mp4, cnt:7
filename:gwanak_400240.mp4, cnt:8
filename:gwanak_400240.mp4, cnt:9
filename:gwanak_400240.mp4, cnt:10
filename:gwanak_400240.mp4, cnt:11
filename:gwanak_400240.mp4, cnt:12
filename:gwanak_400240.mp4, cnt:13
filename:gwanak_400240.mp4, cnt:14
filename:gwanak_400240.mp4, cnt:15
filename:gwanak_400240.mp4, cnt:16
filename:gwanak_400240.mp4, cnt:17
filename:gwanak_400240.mp4, cnt:18
filename:gwanak_400240.mp4, cnt:19
filename:gwanak_400240.mp4, cnt:20
filename:gwanak_400240.mp4, cnt:21
filename:gwanak_400240.mp4, cnt:22
filename:gwanak_400240.mp4, cnt:23
filename:gwanak_400240.mp4, cnt:24
filename:gwanak_400240.mp4, cnt:25
filename:gwanak_400240.mp4, cnt:26
filename:gwanak_400240.mp4, cnt:27
filename:gwanak_400240.mp4, cnt:28
filename:gwanak_400240.mp4, c

In [74]:
X_train = np.array(ds_x).reshape(tot_video_cnt,width,height,1)
y_train = np.array(ds_y).reshape(tot_video_cnt, 1)

In [75]:
X_train.shape

(20, 400, 240, 1)

In [62]:
X_val = np.array(ds_x).reshape(tot_video_cnt,width,height,1)
y_val = np.array(ds_y).reshape(tot_video_cnt, 1)

In [77]:
video_cnt1 = 10
wf.test_dataset(1,20,video_cnt1,'no_fire_1.mp4') # param : 메인변수, 건너뛸 프레임갯수, 캡처화면수, video명

filename:no_fire_1.mp4, cnt:1
filename:no_fire_1.mp4, cnt:2
filename:no_fire_1.mp4, cnt:3
filename:no_fire_1.mp4, cnt:4
filename:no_fire_1.mp4, cnt:5
filename:no_fire_1.mp4, cnt:6
filename:no_fire_1.mp4, cnt:7
filename:no_fire_1.mp4, cnt:8
filename:no_fire_1.mp4, cnt:9
filename:no_fire_1.mp4, cnt:10


In [78]:
X_test = np.array(ts_x).reshape(video_cnt1,width,height,1)
y_test = np.array(ts_y).reshape(video_cnt1, 1)

In [79]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dropout

#2. 모델1 구성하기
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(width, height, 1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.5))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

Using TensorFlow backend.


In [80]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [81]:
# 5. 모델 학습시키기
model.fit(X_train, y_train, epochs=20, batch_size=32)

Epoch 1/20
20/20 [==============================] - 18s - loss: -64.5381 - acc: 0.0000e+00
Epoch 2/20
20/20 [==============================] - 2s - loss: -175.0977 - acc: 0.0000e+00
Epoch 3/20
20/20 [==============================] - 2s - loss: -258.9075 - acc: 0.0000e+00
Epoch 4/20
20/20 [==============================] - 2s - loss: -476.4812 - acc: 0.0000e+00
Epoch 5/20
20/20 [==============================] - 2s - loss: -775.7010 - acc: 0.0000e+00
Epoch 6/20
20/20 [==============================] - 3s - loss: -781.1769 - acc: 0.0000e+00
Epoch 7/20
20/20 [==============================] - 3s - loss: -781.1769 - acc: 0.0000e+00
Epoch 8/20
20/20 [==============================] - 2s - loss: -781.1769 - acc: 0.0000e+00
Epoch 9/20
20/20 [==============================] - 2s - loss: -781.1769 - acc: 0.0000e+00
Epoch 10/20
20/20 [==============================] - 2s - loss: -781.1769 - acc: 0.0000e+00
Epoch 11/20
20/20 [==============================] - 3s - loss: -781.1769 - acc: 0.0000e+

KeyboardInterrupt: 

In [ ]:
# 7. 모델 평가하기
score = model.evaluate(X_test, y_test, batch_size=32)
print(score)